https://www.kaggle.com/competitions/classification-of-oil-and-gas/data
по различным параметрам определять местонахождение залежей нефти и газа: 0: OFFSHORE (морское), 1: ONSHORE (на суше), 2: ONSHORE-OFFSHORE (смешанное)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import f1_score, classification_report
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings('ignore')

train_path = r'C:\study-1\Proga-5\5_ml\train_oil.csv'
test_path = r'C:\study-1\Proga-5\5_ml\oil_test.csv'

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

print(f"Размер тренировочных данных: {train_df.shape}")
print(f"Размер тестовых данных: {test_df.shape}")

# Предобработка данных
def preprocess_data(df, is_train=True, label_encoders=None):
    df_processed = df.copy()
    
    # Создаем копию целевой переменной для тренировочных данных
    if is_train:
        target = df_processed['Onshore/Offshore'].copy()
        #Для тренировочных данных: извлекаем целевую переменную и удаляем ее из признаков
        df_processed = df_processed.drop('Onshore/Offshore', axis=1)
    else:
        target = None
    
    # Преобразуем текстовые метки в числа
    if is_train:
        target_encoded = target.map({'ONSHORE': 1, 'OFFSHORE': 0, 'ONSHORE-OFFSHORE': 2})
    else:
        target_encoded = None
    
    # Список категориальных колонок для кодирования
    categorical_cols = [
        'Field name', 'Reservoir unit', 'Country', 'Region', 'Basin name', 
        'Tectonic regime', 'Operator company', 'Hydrocarbon type', 
        'Reservoir status', 'Structural setting', 'Reservoir period', 'Lithology'
    ]
    
    # Создаем новый энкодер для каждого категориального столбца. обучает энкодер и преобразует данные
    if is_train:
        label_encoders = {}
        for col in categorical_cols:
            if col in df_processed.columns:
                le = LabelEncoder()
                # Заполняем пропуски перед кодированием
                df_processed[col] = df_processed[col].fillna('Unknown')
                df_processed[col] = le.fit_transform(df_processed[col].astype(str))
                label_encoders[col] = le
    else:
        # Используем существующие энкодеры для тестовых данных
        for col in categorical_cols:
            if col in df_processed.columns and col in label_encoders:
                df_processed[col] = df_processed[col].fillna('Unknown')
                try:
                    #только преобразуем данные (не обучаем заново)
                    df_processed[col] = label_encoders[col].transform(df_processed[col].astype(str))
                except ValueError:
                    # Для новых категорий используем -1
                    df_processed[col] = -1
    
    # Обработка числовых колонок
    numeric_cols = ['Latitude', 'Longitude', 'Depth', 'Thickness (gross average ft)', 
                   'Thickness (net pay average ft)', 'Porosity', 'Permeability']
    
    # Заполняем пропуски в числовых колонках
    for col in numeric_cols:
        if col in df_processed.columns:
            #преобразует в числа, нечисловые значения становятся NaN
            df_processed[col] = pd.to_numeric(df_processed[col], errors='coerce')
            if is_train:
                # заполняет пропуски медианным значением столбца
                imputer = SimpleImputer(strategy='median')
                df_processed[col] = imputer.fit_transform(df_processed[[col]]).ravel()
            else:
                imputer = SimpleImputer(strategy='median')
                df_processed[col] = imputer.fit_transform(df_processed[[col]]).ravel()
    
    return df_processed, target_encoded, label_encoders if is_train else (df_processed, target_encoded)

# Предобработка тренировочных данных
X_train_processed, y_train, label_encoders = preprocess_data(train_df, is_train=True)

# Предобработка тестовых данных
X_test_processed, y_test, _ = preprocess_data(test_df, is_train=False, label_encoders=label_encoders)

# Выравнивание колонок
common_cols = list(set(X_train_processed.columns) & set(X_test_processed.columns))
X_train_final = X_train_processed[common_cols]
X_test_final = X_test_processed[common_cols]

# Масштабирование числовых признаков
scaler = StandardScaler()
numeric_cols_final = [col for col in common_cols if col in ['Latitude', 'Longitude', 'Depth', 
                                                          'Thickness (gross average ft)', 
                                                          'Thickness (net pay average ft)', 
                                                          'Porosity', 'Permeability']]

X_train_final[numeric_cols_final] = scaler.fit_transform(X_train_final[numeric_cols_final])
X_test_final[numeric_cols_final] = scaler.transform(X_test_final[numeric_cols_final])

# Разделение тренировочных данных для валидации
X_tr, X_val, y_tr, y_val = train_test_split(X_train_final, y_train, test_size=0.2, 
                                          random_state=42, stratify=y_train)

# Обучение модели
model = RandomForestClassifier(
    n_estimators=200,      
    max_depth=15,          
    min_samples_split=5,  
    min_samples_leaf=2,    
    random_state=42,       
    class_weight='balanced' 
)

model.fit(X_tr, y_tr)

# Предсказание на валидационной выборке 
y_val_pred = model.predict(X_val)

# Оценка модели
f1 = f1_score(y_val, y_val_pred, average='weighted')
print(f"\nF1-score на валидации: {f1:.4f}")
print("\nОтчет классификации:")
print(classification_report(y_val, y_val_pred))

# Обучение на всех тренировочных данных
model_final = RandomForestClassifier(
    n_estimators=200,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    class_weight='balanced'
)

model_final.fit(X_train_final, y_train)

# Предсказание на тестовых данных
test_predictions = model_final.predict(X_test_final)

# Создание файла с предсказаниями
submission = pd.DataFrame({
    'index': range(len(test_predictions)),
    'Onshore/Offshore': test_predictions
})

# Сохранение результатов
output_path = r'C:\study-1\Proga-5\5_ml\submission.csv'
submission.to_csv(output_path, index=False)

# Анализ важности признаков
feature_importance = pd.DataFrame({
    'feature': X_train_final.columns,
    'importance': model_final.feature_importances_
}).sort_values('importance', ascending=False)
